# Priprava okolja

Terminal, zmanjšaj hostname: `PS1='\u:\W$ '`

## Google Compute Engine (GCE)

### Create a GCE Account

<p>A Google Cloud Platform account is required for this course. You can use an existing GCP account or <a target="_blank" href="https://console.cloud.google.com/freetrial">sign up for a new one</a> with a valid Gmail account.</p>

A credit card is required for Google Cloud Platform, but there is no subscription cost, you pay only for resources you use.

### Create a Project

A GCP project is required for this course. You can use an existing GCP project or create a new one.

Your project name maybe different from your project id.

### Enable Compute Engine and Container Engine APIs

In order to create the cloud resources required by this workshop, you will need to enable the following APIs using the Google API Console:
- Compute Engine API
- Container Engine API

### Enable and explore Cloud Shell

Google Cloud Shell provides you with command-line access to computing resources hosted on Google Cloud Platform and is available now in the Google Cloud Platform Console. Cloud Shell makes it easy for you to manage your Cloud Platform Console projects and resources without having to install the Google Cloud SDK and other tools on your system. With Cloud Shell, the Cloud SDK gcloud command and other utilities you need are always available when you need them.

- [Google Cloud Shell documentation](https://cloud.google.com/shell/docs)
- [Google Cloud Shell getting started guide :](https://classroom.udacity.com/courses/ud615/lessons/7826112332/concepts/81473137730923)
    - Preview and deploy a Kubernetes Engine application
    - Preview and deploy an App Engine application
    - Running gcloud commands with Cloud Shell

### [Install the latest Cloud SDK version](https://cloud.google.com/sdk/docs#install_the_latest_cloud_sdk_version)

1. Ustvarimo projekt na Google Cloud
2. Namestimo Google SDK: https://cloud.google.com/sdk/docs/quickstart-debian-ubuntu
    - `gcloud init --console-only`
    - Izberemo račun in projekt
    - Izberemo zone:  ` europe-west3-a`

3. [Accessing Cloud Shell from the command line](https://cloud.google.com/shell/docs/accessing-cloud-shell-with-gcloud)
    - `gcloud alpha cloud-shell ssh`

### Ustavrimo virtulako 

- VM Instances -> Create New Instance
- Izberemo regijo `europe-west3-a`
- Tip: `e2-medium`
- Allow HTTP traffic
- `gcloud compute instances list`

Povezava: `gcloud compute ssh <IME>`

### Git clone

    git clone https://github.com/leon11s/docker-k8s-public.git

### VS Code

Dodamo ključ iz windowsa med autorizirane ključe in se povežemo na virtualko.

# AWS

## Plume - korak 1: Povezava na SSH virtualko

Open an SSH client.
Locate your private key file. The key used to launch this instance is plumetest.pem

Run this command, if necessary, to ensure your key is not publicly viewable (Linux).
- `chmod 400 plumetest.pem`

Connect to your instance `sing its Public DNS:
- `ec2-3-127-245-150.eu-central-1.compute.amazonaws.com`

Example:
- `ssh -i plumetest.pem ubuntu@ec2-3-127-245-150.eu-central-1.compute.amazonaws.com`

## Plume - korak 2: git in VS Code

Prenesemo gradiva: `git clone https://github.com/leon11s/docker-k8s-public`

VS Code:
- Dodamo SSH public ključ naše naprave med avtorizirane ključe na virtualki.
- Vnesemo: `ssh ubuntu@ec2-18-185-137-243.eu-central-1.compute.amazonaws.com`
- Odpremo želeno mapo

## Plume - korak 3: Kubernetes

#### Namestimo AWS CLI.

https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2-linux.html

Prenesejo skripto `install_env.sh`

Dodajo pravice: `chmod +x install_env.sh`
    
Zaženejo: `./install_env.sh`

Skripta:

```bash
#!/bin/bash

echo "Getting data from git..."
git clone https://github.com/leon11s/docker-k8s-public.git

echo "Installing Docker"
sudo apt-get update

sudo apt-get install \
    apt-transport-https \
    ca-certificates \
    curl \
    gnupg-agent \
    software-properties-common

curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -

sudo add-apt-repository \
   "deb [arch=amd64] https://download.docker.com/linux/ubuntu \
   $(lsb_release -cs) \
   stable"

sudo apt-get update
sudo apt-get install -y docker-ce docker-ce-cli containerd.io


echo "Installing Compose"
sudo curl -L "https://github.com/docker/compose/releases/download/1.27.3/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
sudo chmod +x /usr/local/bin/docker-compose

echo "Installing AWS CLI..."
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
sudo apt install unzip -y
unzip awscliv2.zip
sudo ./aws/install --update
rm -r awscliv2.zip aws

sudo systemctl enable docker
sudo groupadd docker
sudo usermod -aG docker $USER
newgrp docker
```

#### Prijava v AWS CLI

https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-quickstart.html

    aws configure

Vstavimo ID in Key za račun v katerega uporabljajo. Credenetiale dobijo na mailu.

Region: `eu-central-1`

Format: `json`

Preverimo indentiteto: ` aws sts get-caller-identity`

#### Namestitev in prijava v kubectl

Prenesejo skripto `k8s_setup.sh`

Dodajo pravice: `chmod +x k8s_setup.sh`
    
Zaženejo: `./k8s_setup.sh`

Skripta:

```bash
#!/bin/bash

echo "Instaling kubectl..."
sudo apt-get update && sudo apt-get install -y apt-transport-https gnupg2
curl -s https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
echo "deb https://apt.kubernetes.io/ kubernetes-xenial main" | sudo tee -a /etc/apt/sources.list.d/kubernetes.list
sudo apt-get update
sudo apt-get install -y kubectl

echo 'source <(kubectl completion bash)' >>~/.bashrc


echo "Enter username: "
read USER

mkdir ~/.kube
CONFIG_FILE=~/.kube/config

echo "Editing config file..."
cat > ${CONFIG_FILE} <<EOL
apiVersion: v1
clusters:
- cluster:
    certificate-authority-data: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUN5RENDQWJDZ0F3SUJBZ0lCQURBTkJna3Foa2lHOXcwQkFRc0ZBREFWTVJNd0VRWURWUVFERXdwcmRXSmwKY201bGRHVnpNQjRYRFRJd01Ea3dNekl4TVRFME0xb1hEVE13TURrd01USXhNVEUwTTFvd0ZURVRNQkVHQTFVRQpBeE1LYTNWaVpYSnVaWFJsY3pDQ0FTSXdEUVlKS29aSWh2Y05BUUVCQlFBRGdnRVBBRENDQVFvQ2dnRUJBTER1CmNSVWplSzI4US92NWNaemdNWnJGZjhPWS82VnBKaVFsazNsSEp0UEp2SUpSbWgvTEdsNHBFd2o1aHVFUTNPT3UKSjhEQjR3N3BBSElkUVB3QXdnWVQrMFpocnFpMzFTWVA4WWJXZ2hSRU15Nzdic2VUc2k3ZmRJbDQxbVhkbWg2TgpCbjJZSDB6YXpDcUFhU0t1K1RRdklNNlp2d2JTaHp1VFBWUUptTVVyQ2kvRUJkWXprb1ZrUG9pSVZpT2JPdU8rCld3OW9oODVFWmFBeFhnekdDTFNDbWlacEJyRnhsRE1XemZuQlBFSFFmRDg3Q0hLVlNhRFBVVU5obmtpaDIweTkKcFlScG43cEVZMUJCWXRpYU5HMnAvbkQxYmxldkJwR3FRS1BGRFdoV3V3WTNLZUpqTEdGMzM1N1JEV3ZVaUh5VgoxYnlDSU1YV0RyUmhweTBEVEpNQ0F3RUFBYU1qTUNFd0RnWURWUjBQQVFIL0JBUURBZ0trTUE4R0ExVWRFd0VCCi93UUZNQU1CQWY4d0RRWUpLb1pJaHZjTkFRRUxCUUFEZ2dFQkFIZkhPWHpqVmhSMU12QTFrNjYzOXplRFBGMmUKY2FnSEREc1RhSjJ5UStKNmt3RVh2Y3YrTit4K01qUW9aaW0vTFBlajF6UEJKd2xqQ0hKQmp4T2VRT0pxd0lkUApOVjY2Tmp3cUdOckFrMFdGcnhKaGdxdUJVcHFNemw1bGsrcm5VMmYyTjZiZ1ZxOEhEMEpwZVoxWHlYUjdScDYyClJwT1krQ1hwMEI2VHFTaU0wTVhJL2tBNnI4REFreUhNMHBidlF5VjU2RllQTnk2MmdNTDlyTjRlTEFSSTRMUEYKNHdYNzdzSllZNHU1TDZobkx4Sit3cGZkOGs5U2xZMUpTTmh5R0RGSGdFK2dxTENHNjFUUSsvbThCMmpzNUxSbgp5ZnRGcW1tdmRSenVrWUpRV3BnLy8zWmJMY0dKcFR5UlFaUER0cXhmM1Yvay92dnJpUDgxU0MzaFNIUT0KLS0tLS1FTkQgQ0VSVElGSUNBVEUtLS0tLQo=
    server: https://196F2E96E3480B39865CA04FD3ABA680.gr7.eu-central-1.eks.amazonaws.com
  name: arn:aws:eks:eu-central-1:641659091182:cluster/icta-plume
contexts:
- context:
    cluster: arn:aws:eks:eu-central-1:641659091182:cluster/icta-plume
    namespace: ${USER}
    user: arn:aws:eks:eu-central-1:641659091182:cluster/icta-plume
  name: arn:aws:eks:eu-central-1:641659091182:cluster/icta-plume
current-context: arn:aws:eks:eu-central-1:641659091182:cluster/icta-plume
kind: Config
preferences: {}
users:
- name: arn:aws:eks:eu-central-1:641659091182:cluster/icta-plume
  user:
    exec:
      apiVersion: client.authentication.k8s.io/v1alpha1
      args:
      - --region
      - eu-central-1
      - eks
      - get-token
      - --cluster-name
      - icta-plume
      - --role-arn
      - arn:aws:iam::641659091182:role/eks-role-${USER}
      command: aws
      env: null

EOL
```

# Priprava Kubernetes za userje

### Admin

Pripravljeno po navodilih: https://aws.amazon.com/premiumsupport/knowledge-center/eks-iam-permissions-namespaces/

- Ustvarimo uporabnika plume-01

- Ustvarimo IAM role, kateremu damo pravico userja, ki želimo, da upravlja z njim

    Pod trusted relationship:
    
    {
      "Version": "2012-10-17",
      "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "AWS": "arn:aws:iam::641659091182:user/plume-01"
          },
          "Action": "sts:AssumeRole"
        }
      ]
    }

    aws iam create-role --role-name Test-Role --assume-role-policy-document trusted_role_test.json --profile administrator

https://docs.aws.amazon.com/eks/latest/userguide/create-kubeconfig.html

- Ustvarimo namespace: `kubectl create namespace plume-01`

```bash
#!/bin/bash

#VARIABLES
CLUSTER_NAME=icta-plume
ACCOUNT=641659091182
GROUP_NAME=PlumeUsers

echo "Enter username: "
read USER
echo "Strting script for user $USER"

# Creating user on AWS
aws iam create-user --user-name ${USER} --profile administrator
aws iam add-user-to-group --user-name ${USER} --group-name ${GROUP_NAME} --profile administrator
aws iam create-access-key --user-name ${USER} --profile administrator > credentials-${USER}.json


echo "Creating AWS role"
cat > temp_role.json <<EOL
{"Version":"2012-10-17","Statement":[{"Effect":"Allow","Principal":{"AWS":"arn:aws:iam::641659091182:user/${USER}"},"Action":"sts:AssumeRole"}]}
EOL

aws iam create-role --role-name eks-role-${USER} --assume-role-policy-document file://temp_role.json --profile administrator

rm temp_role.json

FILE_NAME_ROLE=role-$USER.yaml
FILE_NAME_ROLEBINDING=rolebinding-$USER.yaml

rm ${FILE_NAME_ROLE}
rm ${FILE_NAME_ROLEBINDING}

echo "Creating namespace $USER"
kubectl create namespace ${USER}

echo "Creating role file"
cat > ${FILE_NAME_ROLE} <<EOL
kind: Role
apiVersion: rbac.authorization.k8s.io/v1beta1
metadata:
  name: k8s-${USER}-role
  namespace: ${USER}
rules:
  - apiGroups:
      - ""
      - "apps"
      - "batch"
      - "extensions"
    resources:
      - "configmaps"
      - "cronjobs"
      - "deployments"
      - "events"
      - "ingresses"
      - "jobs"
      - "pods"
      - "pods/attach"
      - "pods/exec"
      - "pods/log"
      - "pods/portforward"
      - "secrets"
      - "services"
      - "replicationcontrollers"
      - "replicationcontrollers/scale"
      - "replicasets"
      - "daemonsets"
      - "endpoints"
      - "statefulsets"
      - "horizontalpodautoscalers"
      - "lables"
      - "annotations"
    verbs:
      - "create"
      - "delete"
      - "describe"
      - "get"
      - "list"
      - "patch"
      - "update"

EOL

kubectl apply -f ${FILE_NAME_ROLE}


echo "Creating rolebinding file"
cat > ${FILE_NAME_ROLEBINDING} <<EOL
kind: RoleBinding
apiVersion: rbac.authorization.k8s.io/v1beta1
metadata:
  name: k8s-${USER}-rolebinding
  namespace: ${USER}
subjects:
- kind: User
  name: k8s-${USER}-user
roleRef:
  kind: Role
  name: k8s-${USER}-role
  apiGroup: rbac.authorization.k8s.io

EOL

kubectl apply -f ${FILE_NAME_ROLEBINDING}

eksctl create iamidentitymapping --cluster ${CLUSTER_NAME} --arn arn:aws:iam::${ACCOUNT}:role/eks-role-${USER} --username  k8s-${USER}-user
```

Namestijo elk-cli

    curl --silent --location "https://github.com/weaveworks/eksctl/releases/latest/download/eksctl_$(uname -s)_amd64.tar.gz" | tar xz -C /tmp
    sudo mv /tmp/eksctl /usr/local/bin
    eksctl version

### Userji

- Skopirajo kubeconfig file
- V file vključijo

      - --role-arn
      - arn:aws:iam::641659091182:role/eks-role-plume-01

    kubectl config set-context --current --namespace=plume-01

In [ ]:
echo 'export KUBECONFIG=$KUBECONFIG:~/.kube/config-devel' >> ~/.bashrc

## Users

     **root** |**Administrators**|**Instructors**

    leon-ltfe |
              | Administrator |
                              | - instructor-leon
                                - instructor-dejan


### Dostop do CLI terminala

### Ustvarimo plume uporabniške račune

## Virtualka

- Install the AWS CLI
- Configure your AWS CLI credentials
- Install and configure kubectl

Virtualko uporabljajo kot dev okolje in v njo namestijo vse potrebno, tudi `kubectl`.

## Kubernetes

### Create cluster

https://docs.aws.amazon.com/eks/latest/userguide/getting-started-console.html

- Create your Amazon EKS cluster IAM role
- Create your Amazon EKS cluster
- Create a kubeconfig file
    - `aws eks --region eu-central-1 update-kubeconfig --name icta-plume`
    - `kubectl get svc`

- Create compute

- `kubectl get nodes --watch`

### Terminal 

### Mi

https://docs.aws.amazon.com/eks/latest/userguide/add-user-role.html

### Oni

Konfiguracija CLI

    aws configure

In [ ]:
https://aws.amazon.com/premiumsupport/knowledge-center/eks-iam-permissions-namespaces/